In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

# Import files & Merge

In [2]:
Steps0 = pd.read_csv("Steps.csv")
Calendar = pd.read_csv("Calendar.csv")
Sleep = pd.read_csv("Sleep_Schedule.csv")
Weather = pd.read_csv("weather.csv")

#Weather['date'] = pd.to_datetime(Weather['date'])

Sleep.columns = Sleep.columns.str.strip()
Calendar.columns = Calendar.columns.str.strip()


#Calendar columns we want 
Weather = Weather[['date','temp_min','temp_max','is_rain']]
Calendar = Calendar[['Date','AD_Month','AH_Month',"Long_Vacation","Short_Vacation","Ramadan","Sch_Eid_Adha","Sch_Eid_Fatr","National_Day","National_Day_Ext","AD_WeekdayNum"]]
Steps0 = Steps0[["Date","Steps"]]
Sleep = Sleep[['Date','Woke_Hour']]



Merged = pd.merge(Steps0,Calendar,how="left",on="Date")
Merged = pd.merge(Merged,Sleep,how='left',on="Date")
Merged = pd.merge(Merged,Weather,how='left',left_on='Date',right_on='date')
Merged.sort_values('Date',inplace=True)
del Merged['date']
Merged['is_rain'] = Merged['is_rain'].astype('int')
Merged = Merged.sort_values("Date")
Merged = Merged.iloc[:,1:]

In [3]:
Merged

,Steps,AD_Month,AH_Month,Long_Vacation,Short_Vacation,Ramadan,Sch_Eid_Adha,Sch_Eid_Fatr,National_Day,National_Day_Ext,AD_WeekdayNum,Woke_Hour,temp_min,temp_max,is_rain
0,3140,6,9,1,0,1,0,0,0,0,5,18,86,111,0
1,9810,6,9,1,0,1,0,0,0,0,6,20,88,113,0
2,6551,6,9,1,0,1,0,0,0,0,0,6,90,111,0
3,6107,6,9,1,0,1,0,0,0,0,1,6,88,113,0
4,2569,6,9,1,0,1,0,0,0,0,2,6,86,113,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,4454,9,2,0,0,0,0,0,0,0,3,3,73,100,0
1203,3702,9,2,0,0,0,0,0,0,0,4,6,73,100,0
1204,3939,9,2,0,0,0,0,0,0,0,5,8,77,104,0
1205,2847,9,2,0,0,0,0,0,0,1,6,4,77,102,0


# Correlation Analysis

In [4]:
## correlation matrix,  and pair plot
Merged.corr()["Steps"]

Steps               1.000000
AD_Month            0.162495
AH_Month           -0.025899
Long_Vacation       0.145277
Short_Vacation     -0.095913
Ramadan            -0.110622
Sch_Eid_Adha             NaN
Sch_Eid_Fatr       -0.081566
National_Day       -0.029345
National_Day_Ext   -0.046646
AD_WeekdayNum      -0.003580
Woke_Hour          -0.003678
temp_min            0.094397
temp_max            0.058535
is_rain            -0.009279
Name: Steps, dtype: float64

# Generate Lag values

In [5]:

for i in range(1,9):
    Merged['Lag'+str(i)] = Merged['Steps'].shift(i)
Merged

Merged = Merged.dropna()

# Normalization

In [6]:
Merged

,Steps,AD_Month,AH_Month,Long_Vacation,Short_Vacation,Ramadan,Sch_Eid_Adha,Sch_Eid_Fatr,National_Day,National_Day_Ext,...,temp_max,is_rain,Lag1,Lag2,Lag3,Lag4,Lag5,Lag6,Lag7,Lag8
8,3811,6,9,1,0,1,0,0,0,0,...,111,0,1756.0,743.0,2283.0,2569.0,6107.0,6551.0,9810.0,3140.0
9,2202,6,9,1,0,1,0,0,0,0,...,109,0,3811.0,1756.0,743.0,2283.0,2569.0,6107.0,6551.0,9810.0
10,11535,6,9,1,0,1,0,0,0,0,...,109,0,2202.0,3811.0,1756.0,743.0,2283.0,2569.0,6107.0,6551.0
11,10248,6,9,1,0,1,0,0,0,0,...,106,0,11535.0,2202.0,3811.0,1756.0,743.0,2283.0,2569.0,6107.0
12,10022,6,9,1,0,1,0,0,0,0,...,104,0,10248.0,11535.0,2202.0,3811.0,1756.0,743.0,2283.0,2569.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,4454,9,2,0,0,0,0,0,0,0,...,100,0,5448.0,2739.0,2668.0,4474.0,4337.0,980.0,2310.0,2378.0
1203,3702,9,2,0,0,0,0,0,0,0,...,100,0,4454.0,5448.0,2739.0,2668.0,4474.0,4337.0,980.0,2310.0
1204,3939,9,2,0,0,0,0,0,0,0,...,104,0,3702.0,4454.0,5448.0,2739.0,2668.0,4474.0,4337.0,980.0
1205,2847,9,2,0,0,0,0,0,0,1,...,102,0,3939.0,3702.0,4454.0,5448.0,2739.0,2668.0,4474.0,4337.0


# Modelling

#  

## -- Cross- Validation

In [12]:

# selected_columns =  Merged.loc[:,["Lag1","Lag2","Lag3","Lag4","Lag5","Lag6","Lag7","Long_Vacation","AD_Month","temp_min"]].copy()


# y = selected_columns.iloc[:,:1]
# x_org = selected_columns.iloc[:,1:]
# selected_columns

KFOLD_TEST(input_df)

,Lag1,Lag2,Lag3,Lag4,Lag5,Lag6,Lag7,Long_Vacation,AD_Month,temp_min
8,1756.0,743.0,2283.0,2569.0,6107.0,6551.0,9810.0,1,6,93
9,3811.0,1756.0,743.0,2283.0,2569.0,6107.0,6551.0,1,6,82
10,2202.0,3811.0,1756.0,743.0,2283.0,2569.0,6107.0,1,6,81
11,11535.0,2202.0,3811.0,1756.0,743.0,2283.0,2569.0,1,6,84
12,10248.0,11535.0,2202.0,3811.0,1756.0,743.0,2283.0,1,6,81
...,...,...,...,...,...,...,...,...,...,...
1202,5448.0,2739.0,2668.0,4474.0,4337.0,980.0,2310.0,0,9,73
1203,4454.0,5448.0,2739.0,2668.0,4474.0,4337.0,980.0,0,9,73
1204,3702.0,4454.0,5448.0,2739.0,2668.0,4474.0,4337.0,0,9,77
1205,3939.0,3702.0,4454.0,5448.0,2739.0,2668.0,4474.0,0,9,77


## -- Regularization

In [7]:

#features = ["AD_Month","AH_Month","Long_Vacation","Short_Vacation","Ramadan","Sch_Eid_Adha","Sch_Eid_Fatr","National_Day","National_Day_ext","AD_Weekdaynum","Woke_Hour"]


y = Merged.iloc[:,:1]
x_org = Merged.iloc[:,1:]
x_org
x = x_org.copy()
#x = x.loc[:,["Lag1","Lag2","Lag3","Lag4","Lag5","Lag6","Lag7","AD_WeekdayNum","Long_Vacation","Short_Vacation","Ramadan","National_Day_Ext","Woke_Hour"]].copy()
x = x.loc[:,["Lag1","Lag2","Lag3","Lag4","Lag5","Lag6","Lag7","Long_Vacation","AD_Month","temp_min"]].copy()

#del x['Sch_Eid_Adha']
#del x['Sch_Eid_Fatr']
#del x['National_Day']
#del x['National_Day_Ext']
#del x['Woke_Hour']
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

lm = LinearRegression()
lm.fit(X_train,y_train)
print(lm.score(X_test,y_test))
#y, X = patsy.dmatrices('Steps ~  AD_WeekdayNum', data=Merged, return_type="dataframe")
mean_absolute_error(lm.predict(X_test),y_test)
# Create your model
#model = sm.OLS(y, X)
#lm.predict(X_test)
# Fit your model to your training set
#fit = model.fit()

# Print summary statistics of the model's performance
#fit.summary()

0.3574665525490285


1813.6642375147615

In [44]:
# loop through each feature and log r squared
rsq = []
mse = []
features = ["is_rain","temp_min","temp_max","AD_Month","AH_Month","Long_Vacation","Short_Vacation","Ramadan","Sch_Eid_Adha","Sch_Eid_Fatr","National_Day","National_Day_Ext","AD_WeekdayNum","Woke_Hour"]
for feature in features:
    x = x_org.copy()
    x = x.loc[:,[feature]].copy()
    X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    rsq.append(lm.score(X_test,y_test))
    mse.append(mean_absolute_error(lm.predict(X_test),y_test))
    #y, X = patsy.dmatrices('Steps ~  AD_WeekdayNum', data=Merged, return_type="dataframe")
    
list(zip(features,rsq))

[('is_rain', -0.0052751593886384285),
 ('temp_min', 0.008070591527205129),
 ('temp_max', 0.0018472179881753314),
 ('AD_Month', 0.006529392232651476),
 ('AH_Month', -0.0239820285048693),
 ('Long_Vacation', 0.028176756272644377),
 ('Short_Vacation', 0.003504125152054116),
 ('Ramadan', 0.008162157896928313),
 ('Sch_Eid_Adha', -0.004284802024396406),
 ('Sch_Eid_Fatr', 0.00370217457120936),
 ('National_Day', -0.0040274452651380965),
 ('National_Day_Ext', -0.001418733192048327),
 ('AD_WeekdayNum', -0.006381166636464819),
 ('Woke_Hour', -0.012088361021947103)]

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
rf = RandomForestRegressor(n_estimators=2000, max_features=3, max_depth=5)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("Random Forest Regression:", r2_score(y_test, y_pred))

gbm = GradientBoostingRegressor(n_estimators=500, max_depth=3, learning_rate=.01)
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)
print("Gradient Boosted Regression:", r2_score(y_test, y_pred))

In [ ]:
Steps = Steps.dropna()
y = Steps.iloc[:,1:2]
x = Steps.iloc[:,2:]
#split 800, the rest 407
X_train = x[:800]
X_test = x[800:]
y_train = y[:800]
y_test = y[800:]

X_test

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)
#print(X_test)
score = lm.score(X_test,y_test)
#y, X = patsy.dmatrices('Steps ~  AD_WeekdayNum', data=Merged, return_type="dataframe")
print(mean_absolute_error(lm.predict(X_test),y_test))
print(score)

In [ ]:
lm.predict(pd.DataFrame(columns=['Lag1'],data=[7]))